In [1]:
import time
import os
import sys
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchmetrics import PeakSignalNoiseRatio

from tqdm import tqdm

#from torch.utils.tensorboard import SummaryWriter
# import scipy.io
# from utils.utils_metric import batch_PSNR, batch_SSIM

import options 

from datasets.main_dataset import get_dataloaders
from models.DVBCDModel import DVBCDModel

print(torch.__version__)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)

USE_GPU = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if USE_GPU and torch.cuda.is_available():
    DEVICE = torch.device('cuda:3')
else:
    DEVICE = torch.device('cpu')
print('using device:', DEVICE)
#args = set_opts()
args = options.set_opts_jp()
for arg in vars(args):
    print('{:<25s}: {:s}'.format(arg, str(getattr(args, arg))))

1.12.1
using device: cuda:3
data_path                : /data/BasesDeDatos/Camelyon/Camelyon17/training/patches_224/
log_dir                  : ./log
save_model_dir           : ./weights/
save_model_freq          : 20
batch_size               : 16
patch_size               : 128
epochs                   : 5
pretraining_epochs       : 1
num_workers              : 64
val_prop                 : 0.1
lr_C                     : 0.0001
lr_M                     : 0.0001
gamma                    : 0.1
clip_grad_C              : 10000.0
clip_grad_M              : 100000.0
sigmaRui_h_sq            : 0.001
sigmaRui_e_sq            : 0.001
theta                    : 0.5
pre_kl                   : 100.0
pre_mse                  : 0.01
CNet                     : unet_6
MNet                     : resnet_18_in


In [2]:
#args.train_data_path = '/data/BasesDeDatos/Camelyon/Camelyon17/training/Toy/'
args.data_path = '/data/BasesDeDatos/Camelyon/Camelyon17/training/patches_224/'

#dataloaders = get_dataloaders(args.data_path, args.patch_size, args.batch_size, args.num_workers, val_prop=0.3)
dataloaders = get_dataloaders(args.data_path, 224, 16, 64, val_prop=0.2)

train_dataloader = dataloaders['train']
val_dataloader = dataloaders['val']
print(len(train_dataloader), len(val_dataloader))
test_dataloader = dataloaders['test']

Available patches: 1247945
Available patches: 1813651
62398 15600


In [3]:
model = DVBCDModel(args, DEVICE)
model.fit(5, train_dataloader, val_dataloader, pretraining=True)

Epoch 1


Epoch 1 - Training:   0%|          | 64/62398 [00:39<9:47:32,  1.77it/s, train_loss=8.29e+3, train_loss_mse=141, train_loss_kl=82.9]       